# Lower Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ x := L x $ where $ L $ is lower triangular.  Vector $ y $ is not to be touched (and, indeed, not even passed into the routines).  This is a little trickier than you might think.  Order matters, since you don't want to corrupt values of $ x $ that are still needed.  Indeed, you may want to do a few small problems by hand if you don't get the right answer.  Also, PictureFLAME may help you see what is going on.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm that takes dot products

## The `Trmv_ln_unb_var1!( L, x )`

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := L x $.  The "_un_" in the name of the routine indicates this is the "lower, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.scal!( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

<h1> <font color=red> NOTE: The algorithms for this operation march from bottom-right to top-left!!! </font> </h1>

In [1]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trmv_ln_unb_var1!(L, x)

    LTL, LTR, 
    LBL, LBR  = flame.part_2x2(L, 
                               0, 0, "BR")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "BOTTOM")

    while size(LBR, 1) < size(L, 1)

        L00,  l01,      L02,  
        l10t, lambda11, l12t, 
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, 
                                                        LBL, LBR, 
                                                        1, 1, "TL")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "TOP")

        #------------------------------------------------------------#

        laff.scal!( lambda11, chi1 )
        laff.dots!( l10t, x0, chi1 )

        #------------------------------------------------------------#

        LTL, LTR, 
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  
                                               l10t, lambda11, l12t, 
                                               L20,  l21,      L22,  
                                               "BR")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "BOTTOM")

    end

    flame.merge_2x1!(xT, 
                     xB, x)

end

Trmv_ln_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [13]:
L = rand(4, 4)
x = rand(4)
xold = rand(4)
laff.copy!(x, xold)

# Notice that L is not lower triangular.  We will only use the lower triangular part.

println( "L before =" )
L

L before =


4×4 Array{Float64,2}:
 0.537314   0.580936  0.101353   0.00630569
 0.360352   0.281655  0.0742317  0.948244  
 0.0782696  0.259455  0.406168   0.312105  
 0.694981   0.729     0.0318094  0.647208  

In [14]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.8934808881309593 
 0.5682685705921706 
 0.6671068071894872 
 0.37423563646800306

In [15]:
Trmv_ln_unb_var1!( L, x )

println( "x after =" )
x

x after =


4-element Array{Float64,1}:
 0.48007959814861684
 0.48202319163423946
 0.48832953813085955
 1.298648643813022  

In [16]:
using LinearAlgebra
LowerTriangular( L ) * xold

4-element Array{Float64,1}:
 0.48007959814861684
 0.48202319163423946
 0.4883295381308595 
 1.298648643813022  

In [17]:
println( "x - ( LowerTriangular( L ) * xold ) = " ) # LowerTriangular() makes a matrix lower triangular
x - ( LowerTriangular( L ) * xold ) 

x - ( LowerTriangular( L ) * xold ) = 


4-element Array{Float64,1}:
 0.0                  
 0.0                  
 5.551115123125783e-17
 0.0                  

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The `Trmv_ln_unb_var2!( L, x )` routine

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := L x $.  The "_ln_" in the name of the routine indicates this is the "lower triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
<li> <code> laff.scal!( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [18]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trmv_ln_unb_var2!(L, x)

    LTL, LTR, 
    LBL, LBR  = flame.part_2x2(L, 
                               0, 0, "BR")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "BOTTOM")

    while size(LBR, 1) < size(L, 1)

        L00,  l01,      L02,  
        l10t, lambda11, l12t, 
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, 
                                                        LBL, LBR, 
                                                        1, 1, "TL")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "TOP")

        #------------------------------------------------------------#

        laff.axpy!( chi1, l21, x2 )
        laff.scal!( lambda11, chi1 )

        #------------------------------------------------------------#

        LTL, LTR, 
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  
                                               l10t, lambda11, l12t, 
                                               L20,  l21,      L22,  
                                               "BR")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "BOTTOM")

    end

    flame.merge_2x1!(xT, 
                     xB, x)

end

Trmv_ln_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [19]:
L = rand(4, 4)
x = rand(4)
xold = rand(4)
laff.copy!(x, xold)

# Notice that L is not lower triangular.  We will only use the lower triangular part.

println( "L before =" )
L

L before =


4×4 Array{Float64,2}:
 0.263821   0.625739   0.0469045  0.684802
 0.0746397  0.964173   0.2556     0.310802
 0.703699   0.0555006  0.275488   0.795835
 0.256547   0.965193   0.171865   0.771629

In [20]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.20532106890358626
 0.4590191228147402 
 0.13517161004176637
 0.0979460470541198 

In [21]:
Trmv_ln_unb_var2!( L, x )

println( "x after =" )
x

x after =


4-element Array{Float64,1}:
 0.0541680924273256 
 0.45789908142175906
 0.20719817446388616
 0.5945258309973844 

In [22]:
using LinearAlgebra
LowerTriangular( L ) * xold

4-element Array{Float64,1}:
 0.0541680924273256 
 0.45789908142175906
 0.20719817446388616
 0.5945258309973844 

In [23]:
println( "x - ( LowerTriangular( L ) * xold ) = " ) # LowerTriangular() makes a matrix lower triangular
x - ( LowerTriangular( L ) * xold ) 

x - ( LowerTriangular( L ) * xold ) = 


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.